# Problem Set 2 - Part 2: Inference

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the following instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

In this part you will work on a problem from the FraCaS test suite. The goal of this problem set is to extend the grammar for Cooper storage from the previous part, apply the sentences to a theorem prover and a model builder, analyse their results and conclude about the inference. Since every sentence may be translated to several formulae there will also be several inference steps and thus several results.



### Pre-requisite knowledge

From `problem-set-1`:
- First order logic
- Lambda calculus
- Feature unification context free grammar

From `problem-set-2-part1.ipynb`:
- Cooper storage



In [1]:
# This task requires NLTK and Jupyter Notebook (IPython package).
import nltk
from nltk.grammar import FeatureGrammar
from nltk.sem import cooper_storage as cs

from utils import display_latex, display_translation, display_tree, display, Markdown
from utils2 import sem_parser, evaluate_sentences, syntax, syntax_notv

read_expr = nltk.sem.Expression.fromstring


### 1.1 Extending the grammar [6 marks]

Extend the grammar below so that it covers the sentences of the inference problem based on FraCaS problem 057.

Several solutions are possible, also some that would not result in entailment. Our lexical knowledge about words leads us how to formalise them in first order logic. Therefore, make sure that you handle the PP attachment in a away that you will be able to show entailments between examples.

In [2]:
sentences = [
    'a Portuguese delegate published the result on climate',
    'a Portuguese delegate published a result',
    'a Portuguese delegate published the result',
    'a Portuguese published a result',
]

In [3]:
fcfg_storage_base = r"""
% start S

S[SEM=[CORE=<?subj(?vp)>, STORE=(?b1+?b2)]] -> NP[NUM=?n,SEM=[CORE=?subj, STORE=?b1]] VP[NUM=?n,SEM=[CORE=?vp, STORE=?b2]]

Nom[NUM=?n,SEM=?s] -> N[NUM=?n,SEM=?s]

VP[NUM=?n,SEM=?s] -> IV[NUM=?n,SEM=?s]
VP[NUM=?n,SEM=[CORE=<?v(?obj)>, STORE=(?b1+?b2)]] -> TV[NUM=?n,SEM=[CORE=?v, STORE=?b1]] NP[SEM=[CORE=?obj, STORE=?b2]]
VP[NUM=?n,SEM=[CORE=<?v(?pp)(?obj)>, STORE=(?b1+?b2+?b3)]] -> DTV[NUM=?n,SEM=[CORE=?v, STORE=?b1]] NP[SEM=[CORE=?obj, STORE=?b2]] PP[+TO,SEM=[CORE=?pp, STORE=?b3]]

PP[+TO, SEM=[CORE=?np, STORE=?b1]] -> P[+TO] NP[SEM=[CORE=?np, STORE=?b1]]
"""

fcfg_storage_lexicon = r"""
PropN[NUM=sg,SEM=[CORE=<\P.P(angus)>, STORE=(/)]] -> 'Angus'
PropN[NUM=sg,SEM=[CORE=<\P.P(cyril)>, STORE=(/)]] -> 'Cyril'
PropN[NUM=sg,SEM=[CORE=<\P.P(irene)>, STORE=(/)]] -> 'Irene'

Det[NUM=sg,SEM=[CORE=<\P Q.all x.(P(x) -> Q(x))>, STORE=(/)]] -> 'every'
Det[NUM=sg,SEM=[CORE=<\P Q.exists x.(P(x) & Q(x))>, STORE=(/)]] -> 'a'

N[NUM=sg,SEM=[CORE=<\x.library(x)>, STORE=(/)]] -> 'library'
N[NUM=sg,SEM=[CORE=<\x.girl(x)>, STORE=(/)]] -> 'girl'
N[NUM=sg,SEM=[CORE=<\x.boy(x)>, STORE=(/)]] -> 'boy'
N[NUM=sg,SEM=[CORE=<\x.book(x)>, STORE=(/)]] -> 'book'

IV[NUM=sg,SEM=[CORE=<\x.smile(x)>, STORE=(/)],TNS=pres] -> 'smiles' 

TV[NUM=sg,SEM=[CORE=<\X x.X(\y.read(x,y))>, STORE=(/)],TNS=pres] -> 'reads'

DTV[NUM=sg,SEM=[CORE=<\Y X x.X(\z.Y(\y.give(x,y,z)))>, STORE=(/)],TNS=pres] -> 'gives'

P[+to] -> 'to'
""" 

fcfg_storage_np = r"""
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?np, @x)>+?b1)]] -> PropN[NUM=?n,SEM=[CORE=?np, STORE=?b1]]
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?det(?nom), @x)>+?b1+?b2)]] -> Det[NUM=?n,SEM=[CORE=?det, STORE=?b1]] Nom[NUM=?n,SEM=[CORE=?nom, STORE=?b2]]
"""

The following code resolves the readings from the Cooper storage. We collect the readings in a dictionary where the key is a sentence string and the value is a list of readings.

In [4]:
# your answers
fcfg_storage_answers_1 = r"""
### Replace X with their proper representations
N[NUM=sg, SEM=[CORE=<\x.portugese(x)>, STORE=(/)]] -> 'Portuguese'
Adj[NUM=sg, SEM=[CORE=<\x.portugese(x)>, STORE=(/)]] -> 'Portuguese'
N[NUM=sg, SEM=[CORE=<\x.delegate(x)>, STORE=(/)]] -> 'delegate'
TV[NUM=sg, SEM=[CORE=<\X.\x. X(\y.publish(x,y))>, STORE=(/)], TNS=past] -> 'published'
P1[+LOC,   SEM=[CORE=<\P.\X.\Q. (X (\y. exists x.(Q(y) & P(x) & on(x,y))))>, STORE=(/)]] -> 'on'
# P1[+LOC,   SEM=[CORE=<\P.\Q.\X. X(\y. exists x.(Q(x) & P(y) & on(y,x)))>, STORE=(/)]] -> 'on'
Det[NUM=?n, SEM=[CORE=<\P.\Q. exists x.((P(x) & Q(x)) & all y.(P(y) -> (x = y)))>, STORE=(/)]] -> 'the'
N[NUM=sg, SEM=[CORE=<\x.result(x)>, STORE=(/)]] -> 'result'
N[NUM=sg, SEM=[CORE=<\x.climate(x)>, STORE=(/)]] -> 'climate'

## Extend the other missig rules too
Nom[NUM=?n,SEM=[CORE=<\P. (?adj(P) & ?nom(P))>, STORE=(?b1+?b2)]] -> Adj[NUM=?n,SEM=[CORE=?adj, STORE=?b1]] Nom[NUM=?n,SEM=[CORE=?nom, STORE=?b2]]

# PP[SEM=[CORE=<(?p(?nom1))(?nom2)>, STORE=(?b1+?b2+?b3)]] -> Nom[LOC=?l, SEM=[CORE=?nom1, STORE=?b1]] P1[LOC=?l, SEM=[CORE=?p, STORE=?b2]] Nom[LOC=?l, SEM=[CORE=?nom2, STORE=?b3]]
PP[SEM=[CORE=<?p(?nom)>, STORE=(?b1+?b2)]] -> P1[LOC=?l, SEM=[CORE=?p, STORE=?b1]] Nom[LOC=?l, SEM=[CORE=?nom, STORE=?b2]] 
# NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?pp(?det), @x)>+?b1+?b2)]] -> Det[NUM=?n,SEM=[CORE=?det, STORE=?b1]] PP[NUM=?n,SEM=[CORE=?pp, STORE=?b2]]
NP[NUM=?n,SEM=[CORE=<?pp(?np)>, STORE=(?b1+?b2)]] -> NP[NUM=?n,SEM=[CORE=?np, STORE=?b1]] PP[NUM=?n,SEM=[CORE=?pp, STORE=?b2]]
"""

# this is going to add new rules to the syntax:
fcfg_storage = fcfg_storage_base + fcfg_storage_np + fcfg_storage_lexicon + fcfg_storage_answers_1
new_syntax = FeatureGrammar.fromstring(fcfg_storage)

In [5]:
sentence_readings = sem_parser(sentences, new_syntax, verbose=False, is_cs=True)

# print all readings
for i, (sent, semreps) in enumerate(sentence_readings.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)

1. a Portuguese delegate published the result on climate


"1": $\exists\ x.(result(x)\ \land\ \exists\ z_{2}.(portugese(z_{2})\ \land\ delegate(z_{2})\ \land\ \exists\ z_{1}.(publish(z_{2},x)\ \land\ climate(z_{1})\ \land\ on(z_{1},x)))\ \land\ \forall\ y.(result(y)\ \rightarrow\ (x\ =\ y)))$

"2": $\exists\ x.(portugese(x)\ \land\ delegate(x)\ \land\ \exists\ z_{3}.(result(z_{3})\ \land\ \exists\ z_{1}.(publish(x,z_{3})\ \land\ climate(z_{1})\ \land\ on(z_{1},z_{3}))\ \land\ \forall\ y.(result(y)\ \rightarrow\ (z_{3}\ =\ y))))$

2. a Portuguese delegate published a result


"1": $\exists\ x.(result(x)\ \land\ \exists\ z_{4}.(portugese(z_{4})\ \land\ delegate(z_{4})\ \land\ publish(z_{4},x)))$

...

"1": $\exists\ x.(result(x)\ \land\ \exists\ z_{6}.(portugese(z_{6})\ \land\ delegate(z_{6})\ \land\ publish(z_{6},x))\ \land\ \forall\ y.(result(y)\ \rightarrow\ (x\ =\ y)))$

"2": $\exists\ x.(portugese(x)\ \land\ delegate(x)\ \land\ \exists\ z_{7}.(result(z_{7})\ \land\ publish(x,z_{7})\ \land\ \forall\ y.(result(y)\ \rightarrow\ (z_{7}\ =\ y))))$

4. a Portuguese published a result


"1": $\exists\ x.(result(x)\ \land\ \exists\ z_{8}.(portugese(z_{8})\ \land\ publish(z_{8},x)))$

"2": $\exists\ x.(portugese(x)\ \land\ \exists\ z_{9}.(result(z_{9})\ \land\ publish(x,z_{9})))$

In [6]:
# make sure understanding this data structure:
sentence_readings

{'a Portuguese delegate published the result on climate': [<ExistsExpression exists x.(result(x) & exists z2.(portugese(z2) & delegate(z2) & exists z1.(publish(z2,x) & climate(z1) & on(z1,x))) & all y.(result(y) -> (x = y)))>,
  <ExistsExpression exists x.(portugese(x) & delegate(x) & exists z3.(result(z3) & exists z1.(publish(x,z3) & climate(z1) & on(z1,z3)) & all y.(result(y) -> (z3 = y))))>],
 'a Portuguese delegate published a result': [<ExistsExpression exists x.(result(x) & exists z4.(portugese(z4) & delegate(z4) & publish(z4,x)))>,
  <ExistsExpression exists x.(portugese(x) & delegate(x) & exists z5.(result(z5) & publish(x,z5)))>],
 'a Portuguese delegate published the result': [<ExistsExpression exists x.(result(x) & exists z6.(portugese(z6) & delegate(z6) & publish(z6,x)) & all y.(result(y) -> (x = y)))>,
  <ExistsExpression exists x.(portugese(x) & delegate(x) & exists z7.(result(z7) & publish(x,z7) & all y.(result(y) -> (z7 = y))))>],
 'a Portuguese published a result': [<Ex

### 1.2 Using Prover9 [4 marks in total]

Show which readings of (1) entail a reading of (2). **[1 mark]**

In [7]:
# There is something missing in this code. You will have to fix it, before you can run it.

prover = nltk.Prover9()
# prover.config_prover9(r'/home/guszarzmo@GU.GU.SE/Programs/prover9/LADR-2009-11A/bin')

def apply_theorem_prover(premises, goals):
    for premise in premises:
        for goal in goals:
            print("Premise      : %s" % (premise))
            print("Goal         : %s" % (goal))
            print("Prover result: %s" % (prover.prove(goal, [premise])))
            print(10*'----')
            
mypremises = sentence_readings['a Portuguese delegate published the result on climate']
mygoals = sentence_readings['a Portuguese delegate published a result']
apply_theorem_prover(mypremises, mygoals)

Premise      : exists x.(result(x) & exists z2.(portugese(z2) & delegate(z2) & exists z1.(publish(z2,x) & climate(z1) & on(z1,x))) & all y.(result(y) -> (x = y)))
Goal         : exists x.(result(x) & exists z4.(portugese(z4) & delegate(z4) & publish(z4,x)))
Prover result: True
----------------------------------------
Premise      : exists x.(result(x) & exists z2.(portugese(z2) & delegate(z2) & exists z1.(publish(z2,x) & climate(z1) & on(z1,x))) & all y.(result(y) -> (x = y)))
Goal         : exists x.(portugese(x) & delegate(x) & exists z5.(result(z5) & publish(x,z5)))
Prover result: True
----------------------------------------
Premise      : exists x.(portugese(x) & delegate(x) & exists z3.(result(z3) & exists z1.(publish(x,z3) & climate(z1) & on(z1,z3)) & all y.(result(y) -> (z3 = y))))
Goal         : exists x.(result(x) & exists z4.(portugese(z4) & delegate(z4) & publish(z4,x)))
Prover result: True
----------------------------------------
Premise      : exists x.(portugese(x) & del

Do all the readings of (1) entail all the readings of (2)? If not, why not? **[1 mark]**

**The answer**

Yes, they do. Since (1) is just a more specific version of (2), No matter in which order the quantifiers are resolved, (1) always entails (2), since it just includes additional information, but no contradictory information. 

Which readings of (1) entail a reading of (4). **[1 mark]** 

In [8]:
# There is something missing in this code. You will have to fix it, before you can run it.
mypremises = sentence_readings['a Portuguese delegate published the result on climate']
mygoals = sentence_readings['a Portuguese published a result']
apply_theorem_prover(mypremises,mygoals)

Premise      : exists x.(result(x) & exists z2.(portugese(z2) & delegate(z2) & exists z1.(publish(z2,x) & climate(z1) & on(z1,x))) & all y.(result(y) -> (x = y)))
Goal         : exists x.(result(x) & exists z8.(portugese(z8) & publish(z8,x)))
Prover result: True
----------------------------------------
Premise      : exists x.(result(x) & exists z2.(portugese(z2) & delegate(z2) & exists z1.(publish(z2,x) & climate(z1) & on(z1,x))) & all y.(result(y) -> (x = y)))
Goal         : exists x.(portugese(x) & exists z9.(result(z9) & publish(x,z9)))
Prover result: True
----------------------------------------
Premise      : exists x.(portugese(x) & delegate(x) & exists z3.(result(z3) & exists z1.(publish(x,z3) & climate(z1) & on(z1,z3)) & all y.(result(y) -> (z3 = y))))
Goal         : exists x.(result(x) & exists z8.(portugese(z8) & publish(z8,x)))
Prover result: True
----------------------------------------
Premise      : exists x.(portugese(x) & delegate(x) & exists z3.(result(z3) & exists z1

Do all the readings of (1) entail all the readings of (4)? If not, why not? **[1 mark]**

**The answer**
Yes, For the same reason as above - (4) is even more general than (2), only specifying that the publisher of the results is a Portuguese, but not necessarily a delegate.


### 1.3 Using Mace [2 marks in total]

Show whether (1) entails (3). **[1 mark]**

In [14]:
# There is something missing in this code. 

def apply_model_builder(premises,goals):
    for premise in premises:
        for goal in goals:
            print("Premise : %s" % (premise))
            print("Goal    : %s" % (goal))
            mc1 = nltk.MaceCommand(goal, assumptions=[premise, a1, a2])
            mc2 = nltk.MaceCommand(None, assumptions=[premise, goal, a1, a2])
            mc1.build_model()
            mc2.build_model()
            print("a model-1:")
            print(mc1.valuation)
            print("a model-2:")
            print(mc2.valuation)
            print(10*'...')

mypremises = sentence_readings['a Portuguese delegate published the result on climate']
mygoals = sentence_readings['a Portuguese delegate published a result']
a1 = read_expr('all x. (portugese(x) -> -(climate(x) | result(x)))')
a2 = read_expr('all x. (climate(x) -> -(portugese(x) | result(x)))')
apply_model_builder(mypremises,mygoals)

Premise : exists x.(result(x) & exists z2.(portugese(z2) & delegate(z2) & exists z1.(publish(z2,x) & climate(z1) & on(z1,x))) & all y.(result(y) -> (x = y)))
Goal    : exists x.(result(x) & exists z4.(portugese(z4) & delegate(z4) & publish(z4,x)))
a model-1:
{}
a model-2:
{'C1': 'a',
 'C2': 'b',
 'C3': 'c',
 'C4': 'a',
 'C5': 'b',
 'climate': {('c',)},
 'delegate': {('b',)},
 'on': {('c', 'a')},
 'portugese': {('b',)},
 'publish': {('b', 'a')},
 'result': {('a',)}}
..............................
Premise : exists x.(result(x) & exists z2.(portugese(z2) & delegate(z2) & exists z1.(publish(z2,x) & climate(z1) & on(z1,x))) & all y.(result(y) -> (x = y)))
Goal    : exists x.(portugese(x) & delegate(x) & exists z5.(result(z5) & publish(x,z5)))
a model-1:
{}
a model-2:
{'C1': 'a',
 'C2': 'b',
 'C3': 'c',
 'C4': 'b',
 'C5': 'a',
 'climate': {('c',)},
 'delegate': {('b',)},
 'on': {('c', 'a')},
 'portugese': {('b',)},
 'publish': {('b', 'a')},
 'result': {('a',)}}
..............................

Explain why. **[1 mark]**

**The answer**

First, we have added some informative rules; portugese(), result(), climate() are three different things.

(1) entails (3) for the following reasons:

1. Mace is able to build a model with a goal

2. Mace cannot build a model with the negation of the goal.
    
So the the goal (sentence 3) is consistent with the premises (sentence 1 and the other conditions) and not informative. Hence, the goal (sentence 3) is entailed by the premises (sentence 1).


## Marks

This part of the assignment has a total of 12 marks.